In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, auc, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor  
import warnings
import pickle
import os
import json


warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [21]:
cols = ['ts', 'uid', 'id.orig_h', 'id.orig_p',
        'id.resp_h', 'id.resp_p', 'proto', 'service',
        'duration',  'orig_bytes', 'resp_bytes',
        'conn_state', 'local_orig', 'local_resp',
        'missed_bytes',  'history', 'orig_pkts',
        'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
        'tunnel_parents', 'label']

In [22]:
out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Zeek/timeout1'

df = pd.read_csv(out_dir+"/CIC-IDS-2017_zeek_1.csv")

In [24]:
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,Attack
0,1.499429e+09,C1EAeD458bYVyUesvb,192.168.10.9,1031,192.168.10.3,88,tcp,krb_tcp,0.000707,233,207,RSTR,T,T,0,ShADTdtFar,8,810,8,758,-,Benign
1,1.499429e+09,CaH5jfGIRK2TQfTr4,192.168.10.9,1032,192.168.10.3,88,tcp,krb_tcp,0.000961,313,1532,RSTR,T,T,0,ShADTdtFar,10,1050,8,3408,-,Benign
2,1.499429e+09,C2tUNH28RtqinE3Jt2,192.168.10.9,1033,192.168.10.3,88,tcp,krb_tcp,0.001095,1552,1518,RSTR,T,T,0,ShADaTdtFr,10,3528,10,3460,-,Benign
3,1.499429e+09,CL1bkq1upfPACIrgvk,192.168.10.9,1034,192.168.10.3,88,tcp,krb_tcp,0.000733,1397,1410,RSTR,T,T,0,ShADTdtFar,8,3138,8,3164,-,Benign
4,1.499429e+09,CulAsl4vBzCgRglje7,192.168.10.9,1036,192.168.10.3,88,tcp,krb_tcp,0.000907,1565,1524,RSTR,T,T,0,ShADTadtFr,10,3554,10,3472,-,Benign


In [23]:
df['tunnel_parents'].unique()

array(['-'], dtype=object)

In [10]:
df.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes',
       'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history',
       'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes',
       'tunnel_parents', 'Attack'],
      dtype='object')

In [2]:
def save_scores(timeout, meanScores, stdScore):
    results = {
        'Timeout': timeout,
        'Mean of all scores': meanScores,
        'Std of all Scores': stdScores

    }

    with open(f'../Checkpoints/ET/ET_cic17_zeek_{timeout}.json', 'w') as f:
        json.dump(results, f, indent=4)

In [3]:
def encode(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    les = {}
    for each in cols:
        le_col = LabelEncoder()
        df[each] = le_col.fit_transform(df[each])
        les[each] = le_col
       
    return df, les

In [4]:
timeouts = ['default', 0.5, 1, 2, 3, 4, 5, 6, 10, 30, 60]

# Training

In [6]:
best_f1 = 0
worst_f1 = 1
best_mean, worst_mean, best_std, worst_std = None, None, None, None

save=True

for timeout in timeouts:
    print("Processing timeout : ", timeout)
    if timeout =='default':
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Zeek/{timeout}/CIC-IDS-2017_zeek_{timeout}.csv'
    else:
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Zeek/timeout{timeout}/CIC-IDS-2017_zeek_{timeout}.csv'

    df = pd.read_csv(out_dir)
    
    df = df.drop(columns=['uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents']) # tunnel_parents is empty

    # Handle missing values (if any)
    df.replace({'orig_bytes': '-'}, '0', inplace=True)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce')
    df['orig_bytes'] = df['orig_bytes'].fillna(0).astype('int64')

    df.replace({'resp_bytes': '-'}, '0', inplace=True)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce')
    df['resp_bytes'] = df['resp_bytes'].fillna(0).astype('int64')


    df.replace({'duration': '-'}, '0', inplace=True)
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(0).astype('float64')

    df['service'] = df['service'].replace('-', np.nan)
    df['history'] = df['history'].replace('-', np.nan)

    # Convert categorical variables to numerical using Label Encoding
    # Encode protocol and service types
    label_encoders = {}
    for column in ['proto', 'service', 'conn_state', 'history', 'local_orig', 'local_resp']:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        print('Fold: ', i, 'done!')
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        oe = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
        y_train = oe.fit_transform(y_train.values.reshape(-1, 1))
        y_test = oe.transform(y_test.values.reshape(-1, 1))
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)

        
    meanScores, stdScores = {}, {}
    
    meanScores['f1Mean'] = np.array(f1).mean()
    meanScores['accMean'] = np.array(accuracy).mean()
    meanScores['recMean'] = np.array(recall).mean()
    meanScores['precMean'] = np.array(precision).mean()
    
    stdScores['f1Std'] = np.array(f1).std()
    stdScores['accStd'] = np.array(accuracy).std()
    stdScores['recStd'] = np.array(recall).std()
    stdScores['precStd'] = np.array(precision).std()
    
    print("Mean of all scores: ", meanScores)
    print("Std of all scores: ", stdScores)


    if save:
        save_scores(timeout, meanScores, stdScores)

    if meanScores['f1Mean'] > best_f1: 
        best_timeout = timeout
        best_mean = meanScores
        best_std = stdScores
        best_f1 = meanScores['f1Mean']
    
    if meanScores['f1Mean'] <= worst_f1: 
        
        worst_timeout = timeout
        worst_mean = meanScores
        worst_std = stdScores
        worst_f1 = meanScores['f1Mean']
               
    print('_______________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.6786992930142743, 'accMean': 0.8884808438538654, 'recMean': 0.7001230961274972, 'precMean': 0.6857616133111352}
Std of all scores:  {'f1Std': 0.07009370288718138, 'accStd': 0.026009793102270737, 'recStd': 0.06974058814036661, 'precStd': 0.07067812022841938}
_______________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scores:  {'f1Mean': 0.7188808225105922, 'accMean': 0.8816816845227677, 'recMean': 0.737516399604518, 'precMean': 0.7299121367411219}
Std of all scores:  {'f1Std': 0.059220992575897524, 'accStd': 0.02779048011273995, 'recStd': 0.06577774275561828, 'precStd': 0.05509605539612502}
_______________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
Mean of all scor

In [3]:
best_f1 = 0
best_timeout = None
best_prec = None
best_rec = None
best_acc = None

worst_f1 = 1
worst_acc = None
worst_timeout = None
worst_prec = None
worst_rec = None

for timeout in timeouts:
    print("Processing timeout : ", timeout)
    if timeout =='default':
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Zeek/{timeout}/CIC-IDS-2017_zeek_{timeout}.csv'
    else:
        out_dir = f'/home/meryem.janati/lustre/nlp_team-um6p-st-sccs-id7fz1zvotk/IDS/janati/IDS/Datasets/cicids17/Zeek/timeout{timeout}/CIC-IDS-2017_zeek_{timeout}.csv'

    df = pd.read_csv(out_dir)
    
    df = df.drop(columns=['uid', 'id.orig_h', 'id.resp_h', 'tunnel_parents']) # tunnel_parents is empty

    # Handle missing values (if any)
    df.replace({'orig_bytes': '-'}, '0', inplace=True)
    df['orig_bytes'] = pd.to_numeric(df['orig_bytes'], errors='coerce')
    df['orig_bytes'] = df['orig_bytes'].fillna(0).astype('int64')

    df.replace({'resp_bytes': '-'}, '0', inplace=True)
    df['resp_bytes'] = pd.to_numeric(df['resp_bytes'], errors='coerce')
    df['resp_bytes'] = df['resp_bytes'].fillna(0).astype('int64')


    df.replace({'duration': '-'}, '0', inplace=True)
    df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
    df['duration'] = df['duration'].fillna(0).astype('float64')

    df['service'] = df['service'].replace('-', np.nan)
    df['history'] = df['history'].replace('-', np.nan)

    # Convert categorical variables to numerical using Label Encoding
    # Encode protocol and service types
    label_encoders = {}
    for column in ['proto', 'service', 'conn_state', 'history', 'local_orig', 'local_resp']:
        if column in df.columns:
            le = LabelEncoder()
            df[column] = le.fit_transform(df[column])
            label_encoders[column] = le

    # Split df into features and labels
    X = df.drop(columns=['Attack'])  # Assuming 'label' is the target variable
    y = df['Attack']
    
    accuracy, f1, precision, recall =[], [], [], []
    skf= StratifiedKFold(n_splits=5,random_state=None)
    skf.get_n_splits(X,y)
    
    for (train_index, test_index), i in zip(skf.split(X, y), range(5)):
        X_train,X_test=X.iloc[train_index],X.iloc[test_index]
        y_train,y_test=y.iloc[train_index],y.iloc[test_index]


        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        
        
        # Map unseen values to '<unknown>'
        y_test = y_test.map(lambda s: '<unknown>' if s not in le.classes_ else s)

        # Add '<unknown>' to classes and transform
        le.classes_ = np.append(le.classes_, '<unknown>')
        y_test = le.transform(y_test)
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Initialize and train Extra Trees Classifier
        clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        
        f1Score = f1_score(y_true=y_test, y_pred=pred, average='macro')
        accScore=accuracy_score(y_test, pred)
        precScore = precision_score(y_test, pred, average='macro')
        recScrore = recall_score(y_test, pred, average='macro')
                             
        f1.append(f1Score)
        accuracy.append(accScore)
        precision.append(precScore)
        recall.append(recScrore)
        print('Fold: ', i, 'done!')

        
    f1Mean = np.array(f1).mean()
    if f1Mean > best_f1: 
        best_timeout = timeout
        best_f1 = f1Mean
        #best_report= report
        best_acc = np.array(accuracy).mean()
        best_rec=np.array(recall).mean()
        best_prec=np.array(precision).mean()
    
    if f1Mean <= worst_f1: 
        worst_timeout = timeout
        worst_f1 = f1Mean
        #best_report= report
        worst_acc = np.array(accuracy).mean()
        worst_rec=np.array(recall).mean()
        worst_prec=np.array(precision).mean()
    print('_______________________________________________________')

Processing timeout :  default
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
_______________________________________________________
Processing timeout :  0.5
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
_______________________________________________________
Processing timeout :  1
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
_______________________________________________________
Processing timeout :  2
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
_______________________________________________________
Processing timeout :  3
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
_______________________________________________________
Processing timeout :  4
Fold:  0 done!
Fold:  1 done!
Fold:  2 done!
Fold:  3 done!
Fold:  4 done!
_______________________________________________________
Processing timeout :  5
Fold:  0 done!
Fold:  1 done!
Fold:  2

In [7]:
print("Best Timeout Combination: ", best_timeout)
print("Mean Scores (Best): ", best_mean)
print('Std Scores (Best):', best_std)

Best Timeout Combination:  0.5
Mean Scores (Best):  {'f1Mean': 0.7188808225105922, 'accMean': 0.8816816845227677, 'recMean': 0.737516399604518, 'precMean': 0.7299121367411219}
Std Scores (Best): {'f1Std': 0.059220992575897524, 'accStd': 0.02779048011273995, 'recStd': 0.06577774275561828, 'precStd': 0.05509605539612502}


In [8]:
print("worst Timeout Combination: ", worst_timeout)
print("Mean Scores (Worst): ", worst_mean)
print('Std Scores (Worst):', worst_std)

worst Timeout Combination:  4
Mean Scores (Worst):  {'f1Mean': 0.6677191479517658, 'accMean': 0.8850962703276053, 'recMean': 0.6981484737469615, 'precMean': 0.6767905681281622}
Std Scores (Worst): {'f1Std': 0.05274526526283913, 'accStd': 0.028387945825920347, 'recStd': 0.06441654529120029, 'precStd': 0.04139702013027459}


In [9]:

results = {
    'Best score': {
        'Best Timeout': best_timeout,
        'Mean Scores (Best)': best_mean,
        'Std Scores (Best)': best_std,
    },
    
    'Worst score': {
        'Worst Timeout': worst_timeout,
        'Mean Scores (Worst)': worst_mean,
        'Std Scores (Worst)': worst_std,
    },
    
    'Difference': {
        'Accuracy': (best_mean['accMean'] - worst_mean['accMean'])*100,
        'F1 Score': (best_mean['f1Mean'] - worst_mean['f1Mean'])*100,
        'Precision': (best_mean['precMean'] - worst_mean['precMean'])*100,
        'Recall': (best_mean['recMean'] - worst_mean['recMean'])*100
    }
}



with open('../results/ET_cic17_zeek.json', 'w') as f:
    json.dump(results, f, indent=4)